<a href="https://colab.research.google.com/github/sidudaav/CoronaASA/blob/master/200_Word_Tester_CoronaTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This program calculates MSE for our 200 keywords

In [0]:
!apt-get install libgeos-3.8.0
!apt-get install libgeos-dev
!pip install --upgrade --user git+https://github.com/GeneralMills/pytrends

In [0]:
import pandas as pd                        
from pytrends.request import TrendReq

pytrend = TrendReq()

state_holder = []
increase_holder = []
MSQE_holder = []

In [0]:
def rank_simple(vector):
    return sorted(range(len(vector)), key=vector.__getitem__)

def rankdata(a):
    n = len(a)
    ivec=rank_simple(a)
    svec=[a[rank] for rank in ivec]
    sumranks = 0
    dupcount = 0
    newarray = [0]*n
    for i in range(n):
        sumranks += i
        dupcount += 1
        if i==n-1 or svec[i] != svec[i+1]:
            averank = sumranks / float(dupcount) + 1
            for j in range(i-dupcount+1,i+1):
                newarray[ivec[j]] = averank
            sumranks = 0
            dupcount = 0
    return newarray

In [0]:
case_list = ['NY', 'NJ', 'LA', 'MI', 'CT', 'MA', 'WA', 'IN', 'IL',
             'RI', 'CO', 'GA', 'VT', 'PA', 'NV', 'MD', 'DE', 'MS', 'FL',
             'OK', 'WI', 'KY', 'OH', 'AL', 'MO', 'KS', 'NH', 'VA',
             'CA', 'AZ', 'ID', 'TN', 'SC', 'ME', 'OR', 'IA', 'MN', 'AK', 'ND',
             'TX', 'NM', 'NE', 'NC', 'AR', 'SD', 'MT', 'HI', 'UT',
             'WV', 'WY']

new_case_list = case_list[::-1]

def case_MSQE(in_list):
  rank = rankdata(in_list)
  MSQE = 0
  for i in range(0, 50):
    state = state_list[i]
    if   rank[i] - new_case_list.index(state) - 1:
      MSQE = MSQE + (( rank[i] - new_case_list.index(state) - 1)**2)
      
  MSQE = MSQE / 50

  return MSQE


In [0]:
CaseRank = []
for i in range(0, 50):
  state = state_list[i]
  CaseRank.append(case_list.index(state) + 1)
print(CaseRank)

In [0]:
import time
from time import sleep

kw_list = []
with open('CoronaWords.txt', 'r') as f:
    kw_list = [line.strip() for line in f]
kw_list.pop(0)


sig_lev = None
state_list = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY',
              'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY',
              'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


for kw in kw_list: 
  globals()[kw + '_states'] = []
  globals()[kw + '_increase_states'] = []
  globals()[kw + '_all_significance_level'] = []
  new_kw_list = [kw]
 


  for state in state_list:
    time.sleep(2)
    pytrend.build_payload(new_kw_list, cat=0, timeframe= '2020-01-01 2020-04-05', geo='US-' + state, gprop='')

    df = pytrend.interest_over_time()
    df1 = df[0:31]
    df2 = df[df.shape[0] - 31:df.shape[0]]

    significant_keyword = True

    val_start = df1[kw].mean()
    val_end = df2[kw].mean()

    if val_end >= val_start:
      globals()[kw + '_all_significance_level'].append(val_end / max(val_start, 1))
    if val_start > val_end:
      globals()[kw + '_all_significance_level'].append(val_start / max(1, val_end))

    if val_end > 1.3 * val_start:
      significant_keyword = True
      globals()[kw + '_increase_states'].append(state)
    elif 1.3 * val_end < val_start:
      significant_keyword = True
    else:
      significant_keyword = False
      
    if significant_keyword == True:
      globals()[kw + '_states'].append(state)
  
  print(kw, " MSQE: ", case_MSQE(globals()[kw + '_all_significance_level']))
  state_holder.append(len(globals()[kw + '_states']))
  increase_holder.append(len(globals()[kw + '_increase_states']))
  MSQE_holder.append(case_MSQE(globals()[kw + '_all_significance_level']))

In [0]:
print(MSQE_holder)
print(state_holder)
print(increase_holder)